# Eval file between real and bots votes


In [7]:
import pandas as pd
import numpy as np

def normalize_english_names(df2):
    """
    Step 1: Replace blank spaces with underscores in the english_name column of original
    
    Args:
        df2: DataFrame from original
    
    Returns:
        DataFrame with normalized english_name column
    """
    df2_copy = df2.copy()
    df2_copy['english_name'] = df2_copy['english_name'].str.replace(' ', '_')
    print(f"Step 1: Normalized {len(df2_copy)} english names (spaces → underscores)")
    return df2_copy

def convert_response_to_votes(df1):
    """
    Step 2: Convert Hebrew Response column to voted_pro and voted_aginst columns
    
    Args:
        df1: DataFrame from bot with Name and Response columns
    
    Returns:
        DataFrame with added voted_pro and voted_aginst columns
    """
    df1_copy = df1.copy()
    
    # Initialize the new columns
    df1_copy['voted_pro'] = 0
    df1_copy['voted_aginst'] = 0
    
    # Convert Hebrew responses
    for idx, row in df1_copy.iterrows():
        response = row['Response'].strip() if pd.notna(row['Response']) else ''
        
        if response == 'בעד':  # Pro/For
            df1_copy.at[idx, 'voted_pro'] = 1
            df1_copy.at[idx, 'voted_aginst'] = 0
        elif response == 'נגד':  # Against
            df1_copy.at[idx, 'voted_pro'] = 0
            df1_copy.at[idx, 'voted_aginst'] = 1
        else:
            # Handle unexpected values - set both to 0
            df1_copy.at[idx, 'voted_pro'] = 0
            df1_copy.at[idx, 'voted_aginst'] = 0
            if response:  # Only warn if there's actually a value
                print(f"Warning: Unexpected response '{response}' for {row['Name']}")
    
    pro_count = df1_copy['voted_pro'].sum()
    against_count = df1_copy['voted_aginst'].sum()
    print(f"Step 2: Converted responses - Pro: {pro_count}, Against: {against_count}")
    
    return df1_copy

def analyze_law_passage(df1_converted, df2_normalized):
    """
    Step 3: Analyze if the law passed in both CSVs (more voted_pro than voted_aginst)
    
    Args:
        df1_converted: bot with converted vote columns
        df2_normalized: original with normalized names
    
    Returns:
        Dictionary with analysis results
    """
    # bot analysis
    bot_pro = df1_converted['voted_pro'].sum()
    bot_against = df1_converted['voted_aginst'].sum()
    bot_passed = bot_pro > bot_against
    
    # original analysis
    original_pro = df2_normalized['voted_pro'].sum()
    original_against = df2_normalized['voted_aginst'].sum()
    original_passed = original_pro > original_against
    
    # Agreement check
    both_agree = bot_passed == original_passed
    
    results = {
        'bot': {
            'pro_votes': int(bot_pro),
            'against_votes': int(bot_against),
            'total_votes': len(df1_converted),
            'law_passed': bot_passed
        },
        'original': {
            'pro_votes': int(original_pro),
            'against_votes': int(original_against),
            'total_votes': len(df2_normalized),
            'law_passed': original_passed
        },
        'agreement': {
            'both_agree_on_result': both_agree,
            'both_passed': bot_passed and original_passed,
            'both_failed': not bot_passed and not original_passed
        }
    }
    
    print(f"Step 3: Law passage analysis completed")
    print(f"  bot: {bot_pro} pro, {bot_against} against → {'PASSED' if bot_passed else 'FAILED'}")
    print(f"  original: {original_pro} pro, {original_against} against → {'PASSED' if original_passed else 'FAILED'}")
    print(f"  Agreement: {'YES' if both_agree else 'NO'}")
    
    return results

def find_mismatches(df1_converted, df2_normalized):
    """
    Step 4: Find and print mismatches between the two files for the same names
    
    Args:
        df1_converted: bot with converted vote columns
        df2_normalized: original with normalized names
    
    Returns:
        List of mismatch details
    """
    mismatches = []
    
    # Create lookup dictionary for original using english_name as key
    original_lookup = {}
    for _, row in df2_normalized.iterrows():
        original_lookup[row['english_name']] = {
            'voted_pro': row['voted_pro'],
            'voted_aginst': row['voted_aginst']
        }
    
    # Check each person in bot
    for _, row1 in df1_converted.iterrows():
        name = row1['Name']
        
        if name in original_lookup:
            original_data = original_lookup[name]
            
            # Compare votes
            bot_pro = row1['voted_pro']
            bot_against = row1['voted_aginst']
            original_pro = original_data['voted_pro']
            original_against = original_data['voted_aginst']
            
            # Check for mismatches
            if bot_pro != original_pro or bot_against != original_against:
                mismatch = {
                    'name': name,
                    'bot_pro': bot_pro,
                    'bot_against': bot_against,
                    'original_pro': original_pro,
                    'original_against': original_against,
                    'differences': []
                }
                
                if bot_pro != original_pro:
                    mismatch['differences'].append(f"Pro votes: bot={bot_pro}, original={original_pro}")
                if bot_against != original_against:
                    mismatch['differences'].append(f"Against votes: bot={bot_against}, original={original_against}")
                
                mismatches.append(mismatch)
        else:
            # Name in bot but not in original
            mismatch = {
                'name': name,
                'bot_pro': row1['voted_pro'],
                'bot_against': row1['voted_aginst'],
                'original_pro': 'NOT_FOUND',
                'original_against': 'NOT_FOUND',
                'differences': ['Name exists in bot but not in original']
            }
            mismatches.append(mismatch)
    
    # Check for names in original but not in bot
    bot_names = set(df1_converted['Name'])
    for english_name in original_lookup.keys():
        if english_name not in bot_names:
            original_data = original_lookup[english_name]
            mismatch = {
                'name': english_name,
                'bot_pro': 'NOT_FOUND',
                'bot_against': 'NOT_FOUND',
                'original_pro': original_data['voted_pro'],
                'original_against': original_data['voted_aginst'],
                'differences': ['Name exists in original but not in bot']
            }
            mismatches.append(mismatch)
    
    print(f"Step 4: Found {len(mismatches)} mismatches")
    return mismatches

def categorize_mismatches(mismatches):
    """
    Categorize mismatches into different types
    
    Args:
        mismatches: List of mismatch dictionaries from find_mismatches()
    
    Returns:
        Dictionary with categorized mismatches
    """
    missing_from_bot = []  # Names in original but not in bot
    missing_from_original = []  # Names in bot but not in original
    voting_disagreements = []  # Names in both files but with different votes
    
    for mismatch in mismatches:
        if mismatch['bot_pro'] == 'NOT_FOUND':
            missing_from_bot.append(mismatch['name'])
        elif mismatch['original_pro'] == 'NOT_FOUND':
            missing_from_original.append(mismatch['name'])
        else:
            voting_disagreements.append(mismatch['name'])
    
    return {
        'missing_from_bot': missing_from_bot,
        'missing_from_original': missing_from_original,
        'voting_disagreements': voting_disagreements
    }

def print_mismatch_lists(categorized_mismatches):
    """
    Print organized lists of mismatched names by category
    
    Args:
        categorized_mismatches: Dictionary from categorize_mismatches()
    """
    print("=" * 60)
    print("MISMATCH LISTS BY CATEGORY")
    print("=" * 60)
    
    # Names missing from bot (exist in original only)
    missing_bot = categorized_mismatches['missing_from_bot']
    if missing_bot:
        print(f"\n📋 NAMES MISSING FROM bot ({len(missing_bot)} names):")
        print("   (These names exist in original but not in bot)")
        for i, name in enumerate(missing_bot, 1):
            print(f"   {i:2d}. {name}")
    else:
        print(f"\n📋 NAMES MISSING FROM bot: None")
    
    # Names missing from original (exist in bot only)
    missing_original = categorized_mismatches['missing_from_original']
    if missing_original:
        print(f"\n📋 NAMES MISSING FROM original ({len(missing_original)} names):")
        print("   (These names exist in bot but not in original)")
        for i, name in enumerate(missing_original, 1):
            print(f"   {i:2d}. {name}")
    else:
        print(f"\n📋 NAMES MISSING FROM original: None")
    
    # Names with voting disagreements
    disagreements = categorized_mismatches['voting_disagreements']
    if disagreements:
        print(f"\n🗳️  VOTING DISAGREEMENTS ({len(disagreements)} names):")
        print("   (These names exist in both files but have different votes)")
        for i, name in enumerate(disagreements, 1):
            print(f"   {i:2d}. {name}")
    else:
        print(f"\n🗳️  VOTING DISAGREEMENTS: None")


def main(bot_path, original_path):
    """
    Main function that executes all steps
    """
    # File paths - update these with your actual file paths
         
    try:
        # Load the CSV files
        print("Loading CSV files...")
        df1 = pd.read_csv(bot_path)
        df2 = pd.read_csv(original_path)
        
        # Clean column names (remove extra whitespace)
        df1.columns = df1.columns.str.strip()
        df2.columns = df2.columns.str.strip()
        
        print(f"Loaded bot: {df1.shape[0]} rows")
        print(f"Loaded original: {df2.shape[0]} rows")
        print()
        
        # Step 1: Normalize english names in original
        df2_normalized = normalize_english_names(df2)
        
        # Step 2: Convert Response to vote columns in bot
        df1_converted = convert_response_to_votes(df1)
        
        # Step 3: Analyze law passage
        analysis_results = analyze_law_passage(df1_converted, df2_normalized)
        
        # Step 4: Find mismatches
        mismatches = find_mismatches(df1_converted, df2_normalized)
        
        # Categorize mismatches
        categorized_mismatches = categorize_mismatches(mismatches)
        
        # Print ALL mismatched names first
        print()
        # print_all_mismatched_names(categorized_mismatches)
        
        # Print the organized mismatch lists by category
        print()
        print_mismatch_lists(categorized_mismatches)
        
        # Print the detailed mismatch report
        print()
        # print_mismatches(mismatches)
        
        # Print summary
        print("=" * 60)
        print("SUMMARY")
        print("=" * 60)
        print(f"bot Result: {'PASSED' if analysis_results['bot']['law_passed'] else 'FAILED'} "
              f"({analysis_results['bot']['pro_votes']} pro, {analysis_results['bot']['against_votes']} against)")
        print(f"original Result: {'PASSED' if analysis_results['original']['law_passed'] else 'FAILED'} "
              f"({analysis_results['original']['pro_votes']} pro, {analysis_results['original']['against_votes']} against)")
        print(f"Both files agree: {'YES' if analysis_results['agreement']['both_agree_on_result'] else 'NO'}")
        print(f"Total mismatches: {len(mismatches)}")
        
        return analysis_results, mismatches, categorized_mismatches
        
    except FileNotFoundError as e:
        print(f"Error: Could not find CSV file - {e}")
        print("Please make sure both CSV files exist and update the file paths in the main() function.")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    bot_path = "../../src/חוק לתיקון פקודת המשטרה (מס' 37)_res.csv"
    original_path = "../laws/חוק לתיקון פקודת המשטרה (מס' 37).csv" 
    main(bot_path, original_path)

Loading CSV files...
Loaded bot: 115 rows
Loaded original: 116 rows

Step 1: Normalized 116 english names (spaces → underscores)
Step 2: Converted responses - Pro: 69, Against: 46
Step 3: Law passage analysis completed
  bot: 69 pro, 46 against → PASSED
  original: 61 pro, 55 against → PASSED
  Agreement: YES
Step 4: Found 41 mismatches


MISMATCH LISTS BY CATEGORY

📋 NAMES MISSING FROM bot (1 names):
   (These names exist in original but not in bot)
    1. Yasser_Hujirat

📋 NAMES MISSING FROM original: None

🗳️  VOTING DISAGREEMENTS (40 names):
   (These names exist in both files but have different votes)
    1. Moshe_Abutbul
    2. Gadi_Eisenkot
    3. Yisrael_Eichler
    4. Zeev_Elkin
    5. Moshe_Arbel
    6. Yaakov_Asher
    7. Debbie_Biton
    8. Haim_Biton
    9. Michael_Biton
   10. David_Bitan
   11. Vladimir_Beliak
   12. Ram_Ben_Barak
   13. Avraham_Betzalel
   14. Mai_Golan
   15. Moshe_Gafni
   16. Simon_Davidson
   17. Aryeh_Deri
   18. Boaz_Toporovsky
   19. Yosef_Tayeb


In [ ]:
import pandas as pd
import re
from typing import Dict, List, Tuple, Optional

def load_name_mapping(mapping_file_path: str) -> Dict[str, str]:
    """
    Load the Hebrew to English name mapping from CSV file.
    
    Args:
        mapping_file_path: Path to the names_mapping.csv file
    
    Returns:
        Dictionary mapping Hebrew names to English names
    """
    try:
        df = pd.read_csv(mapping_file_path, encoding='utf-8')
        return dict(zip(df['name'], df['english_name']))
    except Exception as e:
        print(f"Error loading mapping file: {e}")
        return {}

def normalize_name(name: str) -> str:
    """
    Normalize a name by removing extra spaces and special characters.
    
    Args:
        name: Name to normalize
    
    Returns:
        Normalized name
    """
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # Remove extra whitespace and normalize
    normalized = ' '.join(name.strip().split())
    return normalized

def reverse_hebrew_name_order(hebrew_name: str) -> str:
    """
    Convert Hebrew name from "last name, first name" to "first name last name" format.
    This helps with matching when the raw data has names in different order.
    
    Args:
        hebrew_name: Hebrew name in any format
    
    Returns:
        Hebrew name in "first name last name" format
    """
    if not hebrew_name or not isinstance(hebrew_name, str):
        return ""
    
    # If there's a comma, it's likely "last, first" format
    if ',' in hebrew_name:
        parts = [part.strip() for part in hebrew_name.split(',')]
        if len(parts) == 2:
            return f"{parts[1]} {parts[0]}"  # Reverse to "first last"
    
    return hebrew_name

def find_best_match(target_name: str, mapping_dict: Dict[str, str]) -> Optional[str]:
    """
    Find the best match for a target name in the mapping dictionary.
    Tries multiple strategies including exact match, reversed order, and partial matching.
    
    Args:
        target_name: Name to find match for
        mapping_dict: Dictionary of Hebrew to English names
    
    Returns:
        English name if match found, None otherwise
    """
    if not target_name:
        return None
    
    target_normalized = normalize_name(target_name)
    
    # Strategy 1: Exact match
    if target_normalized in mapping_dict:
        return mapping_dict[target_normalized]
    
    # Strategy 2: Try reversed order (for "last, first" format)
    target_reversed = reverse_hebrew_name_order(target_normalized)
    if target_reversed in mapping_dict:
        return mapping_dict[target_reversed]
    
    # Strategy 3: Check all keys with different normalizations
    for hebrew_name, english_name in mapping_dict.items():
        hebrew_normalized = normalize_name(hebrew_name)
        hebrew_reversed = reverse_hebrew_name_order(hebrew_normalized)
        
        if (target_normalized == hebrew_normalized or 
            target_normalized == hebrew_reversed or
            target_reversed == hebrew_normalized):
            return english_name
    
    # Strategy 4: Partial matching (if names contain same words)
    target_words = set(target_normalized.split())
    if len(target_words) > 1:  # Only for multi-word names
        for hebrew_name, english_name in mapping_dict.items():
            hebrew_words = set(normalize_name(hebrew_name).split())
            # If most words match
            if len(target_words.intersection(hebrew_words)) >= len(target_words) - 1:
                return english_name
    
    return None

def process_voting_data(raw_data_df: pd.DataFrame, 
                       mapping_file_path: str,
                       hebrew_name_column: str = 'name',
                       voted_pro_values: List[str] = None,
                       voted_against_values: List[str] = None) -> Tuple[pd.DataFrame, List[str]]:
    """
    Process voting data by mapping Hebrew names to English and adding voting columns.
    
    Args:
        raw_data_df: DataFrame containing the raw voting data
        mapping_file_path: Path to the names_mapping.csv file
        hebrew_name_column: Name of column containing Hebrew names
        voted_pro_values: List of values that indicate "voted pro" (e.g., ['בעד', 'כן'])
        voted_against_values: List of values that indicate "voted against" (e.g., ['נגד', 'לא'])
    
    Returns:
        Tuple of (processed_dataframe, list_of_unmatched_names)
    """
    if voted_pro_values is None:
        voted_pro_values = ['בעד', 'כן', 'pro', 'yes', 'voted_pro']
    
    if voted_against_values is None:
        voted_against_values = ['נגד', 'לא', 'against', 'no', 'voted_against']
    
    # Load the mapping
    name_mapping = load_name_mapping(mapping_file_path)
    if not name_mapping:
        raise ValueError("Could not load name mapping file")
    
    # Create a copy of the dataframe
    df = raw_data_df.copy()
    
    # Add new columns
    df['english_name'] = ''
    df['voted_pro'] = ''
    df['voted_against'] = ''
    
    unmatched_names = []
    
    for idx, row in df.iterrows():
        hebrew_name = row[hebrew_name_column] if hebrew_name_column in df.columns else ''
        
        # Find English name match
        english_name = find_best_match(hebrew_name, name_mapping)
        
        if english_name:
            df.at[idx, 'english_name'] = english_name
            
            # Process voting columns - check all columns for voting indicators
            for col in df.columns:
                if col not in [hebrew_name_column, 'english_name', 'voted_pro', 'voted_against']:
                    cell_value = str(row[col]).strip() if pd.notna(row[col]) else ''
                    
                    # Check for pro votes
                    if any(pro_val in cell_value for pro_val in voted_pro_values):
                        current_pro = df.at[idx, 'voted_pro']
                        df.at[idx, 'voted_pro'] = f"{current_pro}, {col}" if current_pro else col
                    
                    # Check for against votes  
                    if any(against_val in cell_value for against_val in voted_against_values):
                        current_against = df.at[idx, 'voted_against']
                        df.at[idx, 'voted_against'] = f"{current_against}, {col}" if current_against else col
        else:
            unmatched_names.append(hebrew_name)
            df.at[idx, 'english_name'] = 'UNMATCHED'
    
    return df, unmatched_names

def analyze_voting_patterns(processed_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a summary analysis of voting patterns.
    
    Args:
        processed_df: DataFrame processed by process_voting_data
    
    Returns:
        Summary DataFrame with voting statistics
    """
    summary_data = []
    
    for idx, row in processed_df.iterrows():
        hebrew_name = row.get('name', '') if 'name' in processed_df.columns else row.iloc[0]
        english_name = row['english_name']
        voted_pro = row['voted_pro']
        voted_against = row['voted_against']
        
        pro_count = len([x for x in voted_pro.split(',') if x.strip()]) if voted_pro else 0
        against_count = len([x for x in voted_against.split(',') if x.strip()]) if voted_against else 0
        
        summary_data.append({
            'hebrew_name': hebrew_name,
            'english_name': english_name,
            'voted_pro_count': pro_count,
            'voted_against_count': against_count,
            'voted_pro_on': voted_pro,
            'voted_against_on': voted_against,
            'total_votes': pro_count + against_count,
            'match_status': 'Matched' if english_name != 'UNMATCHED' else 'Unmatched'
        })
    
    return pd.DataFrame(summary_data)

# Example usage function
def main_processing_example():
    """
    Example of how to use the functions above.
    Replace with your actual file paths and data.
    """
    
    # Example: Load your raw data (replace with actual file path)
    raw_data = pd.read_csv('your_raw_voting_data.csv')
    
    # Process the data
    processed_data, unmatched = process_voting_data(
        raw_data_df=raw_data,
        mapping_file_path='names_mapping.csv',
        hebrew_name_column='name',  # adjust column name as needed
        voted_pro_values=['בעד', 'כן', 'pro'],
        voted_against_values=['נגד', 'לא', 'against']
    )
    
    # Print unmatched names
    if unmatched:
        print("Unmatched names:")
        for name in unmatched:
            print(f"  - {name}")
    else:
        print("All names were successfully matched!")
    
    # Create voting analysis
    voting_summary = analyze_voting_patterns(processed_data)
    
    # Save results
    processed_data.to_csv('processed_voting_data.csv', index=False, encoding='utf-8')
    voting_summary.to_csv('voting_summary.csv', index=False, encoding='utf-8')
    
    print("Functions are ready to use. Uncomment and modify the example code above.")

if __name__ == "__main__":
    main_processing_example()